In [1]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments, AdamW
from sklearn.model_selection import train_test_split
import torch
from torch import nn

In [2]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:0


In [3]:
from tqdm import tqdm
import pandas as pd

In [4]:
df = pd.read_csv("train_data.csv")

In [5]:
df

,review,rating
0,location not palace excellent hotel booke dthe...,4
1,respite definitely not place stay looking ultr...,3
2,stunning truly memorable spot right beach nusa...,4
3,solid business hotel near embassy stayed hotel...,3
4,nice place make sure lock money warning money ...,3
...,...,...
16387,great base explore new york stayed 4 nights en...,4
16388,wonderful advert paris wonderful introduction ...,4
16389,ideal relaxing holdiay rachel jay green liverp...,3
16390,"watch food, husband went resort 4 nights chris...",2


In [6]:
opinions = list(df["review"])

In [7]:
ratings = list(df["rating"])

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer(opinions, padding=True, truncation=True, return_tensors="pt")
tokens.keys()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [9]:
train_tokens, test_tokens, train_labels, test_labels = train_test_split(tokens['input_ids'], ratings, test_size=0.2)
attention_mask_train, attention_mask_test = train_test_split(tokens['attention_mask'], test_size=0.2)

In [10]:
class OpinionsDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

train_dataset = OpinionsDataset(train_tokens, attention_mask_train, train_labels)
test_dataset = OpinionsDataset(test_tokens, attention_mask_test, test_labels)

In [11]:
set(ratings)

{0, 1, 2, 3, 4}

In [12]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(ratings)))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
! pip install accelerate

In [14]:
! pip freeze | grep accelerate

accelerate==0.31.0


In [15]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.244400,1.212663
2,1.253100,1.249927
3,1.245100,1.213663
4,1.171500,1.117094
5,1.048400,1.026518


TrainOutput(global_step=8200, training_loss=1.2149194652278248, metrics={'train_runtime': 7702.4198, 'train_samples_per_second': 8.512, 'train_steps_per_second': 1.065, 'total_flos': 1.725134101065216e+16, 'train_loss': 1.2149194652278248, 'epoch': 5.0})

In [20]:
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json')

In [21]:
from sklearn.metrics import accuracy_score

In [22]:
def compute_accuracy(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    return accuracy


In [23]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

In [25]:
accuracy = compute_accuracy(model, test_loader, device)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.546203110704483


In [29]:
df_test = pd.read_csv("test_data.csv", names=["review"])

In [30]:
df_test

,review
0,great hotel location stayed 4 nts 24th 28th ja...
1,"n't return overall disappointed hotel, no hot ..."
2,great value location desired problem hotel loc...
3,kind helpfull people people kind helpful.we no...
4,absolutely fabulous melia comfortable star hot...
...,...
4094,cockroaches dirty carpeting not consider 10 de...
4095,"ca n't wait return, husband stayed el san juan..."
4096,coming home stay wind chimes inn like coming h...
4097,good hotel great location stayed apsis splendi...


In [31]:
test_reviews = list(df_test["review"])

In [36]:
test_tokens = tokenizer(test_reviews, padding=True, truncation=True, return_tensors="pt")

In [34]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks):
        self.input_ids = input_ids
        self.attention_masks = attention_masks

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx]
        }

    def __len__(self):
        return len(self.input_ids)

In [37]:
test_dataset = TestDataset(test_tokens['input_ids'], test_tokens['attention_mask'])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

In [38]:
def predict_stars(model, dataloader, device):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            inputs = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**inputs)
            logits = outputs.logits
            batch_predictions = torch.argmax(logits, dim=-1).cpu().numpy()
            predictions.extend(batch_predictions)

    return predictions

In [39]:
model.to(device)
predicted_stars = predict_stars(model, test_loader, device)

In [41]:
import numpy as np

In [45]:
for star in predicted_stars:
    star = str(star).split(".")[0]

In [46]:
np.savetxt("predykcje.csv", predicted_stars, delimiter=",")